In [1]:
import folium

In [2]:
%run '../scripts/ETL.py' '../scripts/paths.json'

22/10/07 06:46:30 WARN Utils: Your hostname, mast30034 resolves to a loopback address: 127.0.1.1; using 45.113.234.45 instead (on interface eth0)
22/10/07 06:46:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 06:46:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/07 06:46:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/07 06:46:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/07 06:46:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/07 06:46:31 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


/opt/miniconda3/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


22/10/07 06:47:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [42]:
final_join3.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,suburb,postcodes,long,lat,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021,fraud_probability_consumer,fraud_probability_merchant
null,null,null,null,null,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,32234779638,39.304036732369084,02e36af7-8c70-436...,2021-11-30,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Lorem Ipsum Sodal...,43186523025,florists supplies...,4.47,b,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,43186523025,56.35375319523918,a1d3526c-f212-41f...,2022-06-06,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Pede Blandit LLP,28517327395,"books, periodical...",0.47,e,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,28517327395,276.3597685809581,b90e183b-25d4-4a6...,2022-06-14,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Nullam Suscipit LLC,86388904574,"computers, comput...",4.96,b,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,86388904574,121.9799348513232,4c55adbf-7114-4bd...,2022-05-24,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Phasellus At Limited,76767266140,"furniture, home f...",4.65,b,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,76767266140,59.46778996058449,4e66a3c6-9691-44a...,2022-03-05,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01


In [39]:
final_join3.select("postcodes").distinct().count()

2343

In [40]:
num_transactions_by_postcode = final_join3.groupBy(['postcodes', 'suburb', 'long', 'lat']).count()

In [41]:
# conver transactions location to pandas df
num_transactions_by_postcode_pdf = num_transactions_by_postcode.toPandas()

aus_coords = [-25.2744, 133.7751]
m = folium.Map(aus_coords, tiles='OpenStreetMap', zoom_start=4.5)

for index, row in num_transactions_by_postcode_pdf.iterrows():
    if row['count'] >= 10000:
        marker_color = 'darkred'
        fill_color = 'darkred'
    elif row['count'] >= 5000:
        marker_color = 'red'
        fill_color = 'red'
    elif row['count'] >= 500:
        marker_color = 'darkorange'
        fill_color = 'darkorange'
    elif row['count'] >= 100:
        marker_color = 'orange'
        fill_color = 'orange'
    elif row['count'] <= 50 :
        marker_color = 'yellow'
        fill_color = 'yellow'
    else:
        marker_color='darkpurple'
        fill_color = 'darkpurple'
        
    folium.Circle(
          location=[row['lat'], row['long']],
          popup= 'Number of transactions: ' +str(row['count']),
          tooltip=row['suburb'],
          radius=row['count'],
          color=marker_color,
          fill=True,
          fill_color=fill_color,
       ).add_to(m)
m.save('../plots/bubble_plot_num_transactions_by_location.html')

## Fraud visualisations

In [3]:
%run '../scripts/fraud_feature.py' 

In [4]:
## The following was modified from MAST30034 Tutorial 2
# read in shape file
sf = gpd.read_file("../data/curated/boundaries.shp")
gdf = gpd.GeoDataFrame(sf)
gdf['SA2_code'] = gdf['SA2_code'].astype(int)
geoJSON = gdf[['SA2_code', 'geometry']].drop_duplicates('SA2_code').to_json()

### Locations of fraudulent consumers

In [8]:
# aggregate data by postcode and consumer fraud probability
consumer_fraud_postcodes = model_with_fraud.select('postcodes', 'SA2_code', 'fraud_probability_consumer')
consumer_fraud_postcodes = consumer_fraud_postcodes.groupBy('postcodes', 'SA2_code')\
.agg(F.sum('fraud_probability_consumer').alias('sum_consumer_fraud'))
consumer_fraud_pdf = consumer_fraud_postcodes.toPandas()

In [9]:
# join shape file to consumer fraud data by sa2 code
consumer_df = consumer_fraud_pdf \
    .merge(gdf[['SA2_code', 'geometry']], left_on='SA2_code', right_on='SA2_code')
consumer_df.head(5)

,postcodes,SA2_code,suburb,sum_consumer_fraud,geometry
0,2229,128011602,CARINGBAH,372.069730,"POLYGON ((151.11363 -34.03443, 151.11380 -34.0..."
1,1495,128011602,CARINGBAH,431.526156,"POLYGON ((151.11363 -34.03443, 151.11380 -34.0..."
2,3925,205031091,CAPE WOOLAMAI,325.289427,"MULTIPOLYGON (((145.34407 -38.56161, 145.34400..."
3,3922,205031091,COWES,231.560942,"MULTIPOLYGON (((145.34407 -38.56161, 145.34400..."
4,3923,205031091,RHYLL,395.083454,"MULTIPOLYGON (((145.34407 -38.56161, 145.34400..."


In [24]:
aus_coords = [-25.2744, 133.7751]
consumer_fraud_map = folium.Map(aus_coords, tiles="Stamen Terrain", zoom_start=4.5)

# add average tips
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=consumer_df, # data source
    columns=['SA2_code','sum_consumer_fraud'], # the columns required
    key_on='properties.SA2_code', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Total consumer fraud probability by SA2 region'
)
c.add_to(consumer_fraud_map)
consumer_fraud_map.save('../plots/consumer_fraud_map.html')

### Locations of fraudulent merchants

In [21]:
# aggregate data by postcode and merchant fraud probability
merchant_fraud_postcodes = model_with_fraud.select('postcodes', 'SA2_code', 'fraud_probability_merchant')
merchant_fraud_postcodes = merchant_fraud_postcodes.groupBy('postcodes', 'SA2_code')\
.agg(F.sum('fraud_probability_merchant').alias('sum_merchant_fraud'))
merchant_fraud_postcodes.limit(5)
merchant_fraud_pdf = merchant_fraud_postcodes.toPandas()

In [22]:
# join shape file to consumer fraud data by sa2 code
merchant_df = merchant_fraud_pdf \
    .merge(gdf[['SA2_code', 'geometry']], left_on='SA2_code', right_on='SA2_code')
merchant_df.head(5)

,postcodes,SA2_code,sum_merchant_fraud,geometry
0,3032,206031113,137.681582,"POLYGON ((144.89943 -37.77039, 144.89978 -37.7..."
1,3523,202021029,22.920000,"POLYGON ((144.54552 -36.92777, 144.54539 -36.9..."
2,2012,117031336,139.854504,"POLYGON ((151.20831 -33.88343, 151.20842 -33.8..."
3,1428,117031336,107.545548,"POLYGON ((151.20831 -33.88343, 151.20842 -33.8..."
4,1420,117031336,50.850000,"POLYGON ((151.20831 -33.88343, 151.20842 -33.8..."


In [26]:
aus_coords = [-25.2744, 133.7751]
merchant_fraud_map = folium.Map(aus_coords, tiles="Stamen Terrain", zoom_start=4.5)

# add average tips
d = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=merchant_df, # data source
    columns=['SA2_code','sum_merchant_fraud'], # the columns required
    key_on='properties.SA2_code', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Total merchant fraud probability by SA2 region'
)
d.add_to(merchant_fraud_map)
merchant_fraud_map.save('../plots/merchant_fraud_map.html')

In [18]:
model_with_fraud.limit(5)

merchant_name,merchant_abn,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,suburb,postcodes,long,lat,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021,fraud_probability_consumer,fraud_probability_merchant,tags,cleaned_tags,store_type,category,is_fraud
Eget Mollis LLC,91720867026,6.31,a,Sydney Romero,2940 Amanda Light...,NSW,Female,91720867026,48.32378416624947,d241fe37-7337-4e2...,2021-05-30,19155,28328,HORNSBY WESTFIELD,1635,151.101,-33.7046,121021577,121021577,Hornsby - East,501442596,5874,6068,11943,1,New South Wales,12524,11998,0.01,0.01,"((watch, clock, a...",watch clock and j...,1,Toys and DIY,0
Egestas Rhoncus P...,61481587663,4.08,b,Sydney Romero,2940 Amanda Light...,NSW,Female,61481587663,3.853189963091505,d621ee3a-4422-4fa...,2021-04-13,19155,28328,HORNSBY WESTFIELD,1635,151.101,-33.7046,121021577,121021577,Hornsby - East,501442596,5874,6068,11943,1,New South Wales,12524,11998,0.01,0.01,[(tEnt and awning...,tent and awning s...,3,"Books, Stationary...",0
Suspendisse Dui C...,46804135891,2.93,c,Sydney Romero,2940 Amanda Light...,NSW,Female,46804135891,58.15827536687406,ade7f36f-fc0d-465...,2021-10-15,19155,28328,HORNSBY WESTFIELD,1635,151.101,-33.7046,121021577,121021577,Hornsby - East,501442596,5874,6068,11943,1,New South Wales,12524,11998,0.01,0.01,"((opticians, opti...",opticians optical...,3,"Books, Stationary...",0
Turpis In Incorpo...,61845622676,2.48,c,Sydney Romero,2940 Amanda Light...,NSW,Female,61845622676,5.100655350466067,219c87d3-9739-499...,2021-09-24,19155,28328,HORNSBY WESTFIELD,1635,151.101,-33.7046,121021577,121021577,Hornsby - East,501442596,5874,6068,11943,1,New South Wales,12524,11998,0.01,0.01,"[[gift, card, nov...",gift card novelty...,4,Electronics,0
Ipsum Dolor Sit C...,80324045558,1.47,c,Sydney Romero,2940 Amanda Light...,NSW,Female,80324045558,19.987966184960584,3c25dd39-7a1f-457...,2022-06-12,19155,28328,HORNSBY WESTFIELD,1635,151.101,-33.7046,121021577,121021577,Hornsby - East,501442596,5874,6068,11943,1,New South Wales,12524,11998,0.01,0.01,"[(gift, card, nov...",gift card novelty...,4,Electronics,0
